# API check conf pandora

*This notebook is used to test the new* «check_pipeline_section» *function*.

 ## 1. "Pandora" version : user config with input + pipeline

### Import

In [ ]:
from pandora import import_plugin
from pandora.check_configuration import (
    check_conf,
    get_config_input,
    check_input_section,
    get_config_pipeline,
    check_pipeline_section,
    concat_conf
)
from pandora.state_machine import PandoraMachine
from pandora.img_tools import rasterio_open, get_metadata

import xarray as xr
import pprint

pp = pprint.PrettyPrinter(indent=4)

### Define user config

In [ ]:
user_cfg = {
    "input": 
    {
        "left": {
            "img": "../data/left_rgb.tif",
            "disp": [-60, 0],
        },
        "right": {
            "img": "../data/right_rgb.tif",
        }
    },
    "pipeline":
    {
        "matching_cost": {"matching_cost_method": "zncc", "band": "r"},
        "optimization": {"optimization_method": "sgm"},
        "disparity": {"disparity_method": "wta"},
        "filter": {"filter_method": "median"},
        "refinement": {"refinement_method": "quadratic"},
        "validation": {"validation_method": "cross_checking_accurate"}
    }
}

### Import pandora plugins (sgm)

In [ ]:
import_plugin()

### Instantiate a pandora machine

In [ ]:
pandora_machine = PandoraMachine()

### Run check config

**Check input**

In [ ]:
user_cfg_input = get_config_input(user_cfg)
cfg_input = check_input_section(user_cfg_input)
pp.pprint(cfg_input)

**Check pipeline**

In [ ]:
user_cfg_pipeline = get_config_pipeline(user_cfg)
metadata_left = get_metadata(
    cfg_input["input"]["left"]["img"],
    cfg_input["input"]["left"]["disp"],
)
metadata_right = get_metadata(
    cfg_input["input"]["right"]["img"], 
    cfg_input["input"]["right"]["disp"],
)

In [ ]:
cfg_pipeline = check_pipeline_section(user_cfg_pipeline, metadata_left, metadata_right, pandora_machine)
pp.pprint(cfg_pipeline)

**update config**

In [ ]:
cfg_update = concat_conf([cfg_input, cfg_pipeline])
pp.pprint(cfg_update)

## 2. "CARS" version : user config with only pipeline and xr.Dataset list

### Define user config

In [ ]:
user_cfg_pipeline = {
    "pipeline":
    {
        "matching_cost": {"matching_cost_method": "zncc", "band": "r"},
        "optimization": {"optimization_method": "sgm"},
        "disparity": {"disparity_method": "wta"},
        "filter": {"filter_method": "median"},
        "refinement": {"refinement_method": "quadratic"},
        "validation": {"validation_method": "cross_checking_accurate"}
    }
}

### Instantiate a pandora machine

In [ ]:
pandora_machine = PandoraMachine()

### image paths

In [ ]:
img_left_path = "../data/left_rgb.tif"
img_right_path = "../data/right_rgb.tif"

### Run check config : only pipeline

**Check pipeline**

In [ ]:
metadata_left = get_metadata(img_left_path, (user_cfg["input"]["left"]["disp"][0], user_cfg["input"]["left"]["disp"][1]))
metadata_right = get_metadata(img_right_path, disparity=None)

In [ ]:
cfg_pipeline = check_pipeline_section(user_cfg_pipeline, metadata_left, metadata_right, pandora_machine)
pp.pprint(cfg_pipeline)

## 3. "Pandora" version : user config with all options

### Instantiate a pandora machine

In [ ]:
pandora_machine = PandoraMachine()

### User cfg

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": "../../tests/pandora/left_rgb.tif",
            "disp": [-60, 0],
            "classif": "../../tests/pandora/left_classif.tif",
        },
        "right": {
            "img": "../../tests/pandora/right_rgb.tif",
            "classif": "../../tests/pandora/right_classif.tif"
        }
    },
    "pipeline":
    {
        "matching_cost": {"matching_cost_method": "zncc", "band": "r"},
        "aggregation": {"aggregation_method": "cbca"},
        "optimization": {"optimization_method": "sgm"},
        "disparity": {"disparity_method": "wta"},
        "filter": {"filter_method": "median"},
        "refinement": {"refinement_method": "quadratic"},
        "validation": {"validation_method": "cross_checking_accurate"},
        "multiscale": {"multiscale_method": "fixed_zoom_pyramid", "num_scales": 3}
    }
}

### Run check config 

**Check input**

In [ ]:
user_cfg_input = get_config_input(user_cfg)
cfg_input = check_input_section(user_cfg_input)
pp.pprint(cfg_input)

**Check pipeline**

In [ ]:
user_cfg_pipeline = get_config_pipeline(user_cfg)
metadata_left = get_metadata(
    cfg_input["input"]["left"]["img"],
    cfg_input["input"]["left"]["disp"],
    cfg_input["input"]["left"]["classif"]
)
metadata_right = get_metadata(
    cfg_input["input"]["right"]["img"],
    cfg_input["input"]["right"]["disp"],
    cfg_input["input"]["right"]["classif"]
)

In [ ]:
cfg_pipeline = check_pipeline_section(user_cfg_pipeline, metadata_left, metadata_right, pandora_machine)
pp.pprint(cfg_pipeline)